In [ ]:
import numpy as np
import pandas as pd
import time

from tensorflow import keras
import tensorflow as tf

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import commonmodules as cm
from generalutil import *
import sys

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
uT_filename = "N2H2_2D_VT_process_using_T.xlsx"
uT_xkey, uT_ykey, uT_x_s, uT_y_s, uT_scalerx, uT_scalery, \
    uT_x1map_toreal, uT_f1set, uT_f1list = \
    read_excel_file_and_norm_tfile (uT_filename)
uT_xk = uT_xkey[0]
uT_x1 = list(uT_f1set[uT_xk])[0]
uT_yk = uT_xk.split("_")[0]
uT_f1 = uT_xk.split("_")[1]
uT_f2 = uT_xk.split("_")[2]
uT_removedx = uT_x1map_toreal[uT_xk][uT_x1]

print(uT_x1, uT_removedx, uT_f1, uT_f2, uT_yk)

nT_filename = "N2H2_2D_VT_process.xlsx"
nT_xkey, nT_ykey, nT_x_s, nT_y_s, nT_scalerx, nT_scalery, \
    nT_x1map_toreal, nT_f1set, nT_f1list = \
    read_excel_file_and_norm (nT_filename)
nT_xk = nT_xkey[0]
nT_x1 = list(nT_f1set[nT_xk])[0]
nT_yk = nT_xk.split("_")[0]
nT_f1 = nT_xk.split("_")[1]
nT_f2 = nT_xk.split("_")[2]
nT_removedx = nT_x1map_toreal[nT_xk][nT_x1]

print(nT_x1, nT_removedx, nT_f1, nT_f2, nT_yk)


In [ ]:
uT_train_x, uT_test_x, uT_train_y, uT_test_y = cm.test_train_split (0, [uT_x1], \
                                                        uT_x_s[uT_xk], uT_y_s[uT_yk])
print(uT_train_x.shape, uT_test_x.shape, uT_train_y.shape, uT_test_y.shape)
print(uT_x_s[uT_xk].shape, uT_y_s[uT_yk].shape)

nT_train_x, nT_test_x, nT_train_y, nT_test_y = cm.test_train_split (0, [nT_x1], \
                                                        nT_x_s[nT_xk], nT_y_s[nT_yk])
print(nT_train_x.shape, nT_test_x.shape, nT_train_y.shape, nT_test_y.shape)
print(nT_x_s[nT_xk].shape, nT_y_s[nT_yk].shape)

In [ ]:
plt.rcParams["figure.figsize"] = (18, 10)
ax = plt.axes(projection='3d')
ax.scatter3D(nT_test_x[:,0], nT_test_x[:,1], nT_test_y, c=nT_test_y, cmap='Greens')
ax.scatter3D(nT_train_x[:,0], nT_train_x[:,1], nT_train_y, c=nT_train_y, cmap='Reds')
plt.show()

plt.rcParams["figure.figsize"] = (18, 10)
ax = plt.axes(projection='3d')
ax.scatter3D(uT_test_x[:,0], uT_test_x[:,1], uT_test_y, c=uT_test_y, cmap='Greens')
ax.scatter3D(uT_train_x[:,0], uT_train_x[:,1], uT_train_y, c=uT_train_y, cmap='Reds')
plt.show()

In [ ]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [ ]:
uT_models = []
uT_model = cm.build_model_GP_1 (uT_train_x, uT_train_y)
uT_models.append(uT_model)
for nu in [1.5, 2.0, 2.5, 3.0]:
    uT_model = cm.build_model_GP_2 (uT_train_x, uT_train_y, nuval = nu)
    uT_models.append(uT_model)
    uT_model = cm.build_model_GP_3 (uT_train_x, uT_train_y, nuval = nu)
    uT_models.append(uT_model)

In [ ]:
uT_test_x_sp = uT_scalerx[uT_xk].inverse_transform(uT_test_x)
uT_test_y_sb = uT_scalery[uT_yk].inverse_transform(uT_test_y)
plt.rcParams["figure.figsize"] = (18, 10)
ax = plt.axes(projection='3d')
ax.scatter3D(uT_test_x_sp[:,0], uT_test_x_sp[:,1], uT_test_y_sb, c=uT_test_y_sb, cmap='Greens')
for udx, model in enumerate(uT_models):
    uT_pred_y = model.predict(uT_test_x, return_std=False)
    uT_pred_y_sb = uT_scalery[uT_yk].inverse_transform(uT_pred_y.reshape(-1, 1))
    uT_testmse = metrics.mean_absolute_error(uT_test_y_sb, uT_pred_y_sb)
    uT_testr2 = metrics.r2_score(uT_test_y_sb, uT_pred_y_sb)
    print("Model ", udx)
    print("Test MSE: ", uT_testmse)
    print("Test R2:  ", uT_testr2)
    ax.scatter3D(uT_test_x_sp[:,0], uT_test_x_sp[:,1], \
                 uT_pred_y_sb, c='r')
plt.show()

In [ ]:
nT_models = []
nT_model = cm.build_model_GP_1 (nT_train_x, nT_train_y)
nT_models.append(nT_model)
for nu in [1.5, 2.0, 2.5, 3.0]:
    nT_model = cm.build_model_GP_2 (nT_train_x, nT_train_y, nuval = nu)
    nT_models.append(nT_model)
    nT_model = cm.build_model_GP_3 (nT_train_x, nT_train_y, nuval = nu)
    nT_models.append(nT_model)

In [ ]:
nT_test_x_sp = nT_scalerx[nT_xk].inverse_transform(nT_test_x)
nT_test_y_sb = nT_scalery[nT_yk].inverse_transform(nT_test_y)
plt.rcParams["figure.figsize"] = (18, 10)
ax = plt.axes(projection='3d')
ax.scatter3D(nT_test_x_sp[:,0], nT_test_x_sp[:,1], nT_test_y_sb, c=nT_test_y_sb, cmap='Greens')
for ndx, model in enumerate(nT_models):
    nT_pred_y = model.predict(nT_test_x, return_std=False)
    nT_pred_y_sb = nT_scalery[nT_yk].inverse_transform(nT_pred_y.reshape(-1, 1))
    nT_testmse = metrics.mean_absolute_error(nT_test_y_sb, nT_pred_y_sb)
    nT_testr2 = metrics.r2_score(nT_test_y_sb, nT_pred_y_sb)
    print("Model ", ndx)
    print("Test MSE: ", nT_testmse)
    print("Test R2:  ", nT_testr2)
    ax.scatter3D(nT_test_x_sp[:,0], nT_test_x_sp[:,1], \
                 nT_pred_y_sb, c='r')
plt.show()